In [1]:
import tkinter as tk
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
"""Backend"""
df = pd.read_csv("ml-100k/u.data",sep='\t',
                 names=['user_id','item_id','rating','ts'])

cols = "item_id|title| release date | video release date |\
              IMDb URL | unknown | Action | Adventure | Animation |\
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |\
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |\
              Thriller | War | Western |"
movies = pd.read_csv('ml-100k/u.item',sep='|',names=cols.split('|'))
data = pd.merge(df.drop('ts',axis=1),movies[['item_id','title']],on='item_id')

avg_rates = data.groupby('title')['rating'].mean()
rate_count = data.groupby('title')['rating'].count()

df_pivot = data.pivot_table(index='user_id',columns='title',
                           values='rating')
"""Backend"""

app = tk.Tk()
app.geometry('600x400')
app.title('Recommendation Engine')


list_box = tk.Listbox(app, height=20,width=30,activestyle='dotbox', fg='black')
list_box.place(x=20,y=20)
for i,t in enumerate(data['title'].unique()):
    list_box.insert(i+1,t)

def recommend():
    inp = list_box.get(list_box.curselection())
    sim_inp = df_pivot.corrwith(df_pivot[inp])
    sim_df = pd.DataFrame(sim_inp,columns=['Correlation'])
    sim_df['Count'] = rate_count    
    sim_df['avg_rate'] = avg_rates
    recom = sim_df[(sim_df['Count'] > 100)].sort_values(by='Correlation',ascending=False).head(10)
    recom.drop(inp,axis=0,inplace=True,errors='ignore')
    recommended = recom.index[:3]    
    result1.set(recommended[0])
    result2.set(recommended[1])
    result3.set(recommended[2])
   
    
tk.Button(app,text='Show Recommendation',command=recommend).place(x=250,y=30)

result1 = tk.Variable(app)
result1.set('Result 1')
tk.Label(app,textvariable=result1).place(x=250,y=100)

result2 = tk.Variable(app)
result2.set('Result 2')
tk.Label(app,textvariable=result2).place(x=250,y=130)

result3 = tk.Variable(app)
result3.set('Result 3')
tk.Label(app,textvariable=result3).place(x=250,y=160)

app.mainloop()